<a href="https://colab.research.google.com/github/WesleyAldridge/HW3_MachineLearning/blob/master/HW3_MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 - Data Visualization For cats and dogs data set using convnet VGG19

###Instructions from professor:

"The goal of this homework is to learn how to leverage pretrained convnets and to use some visualization techniques. You will work with the data set cats vs dogs and use the pretrained convnet VGG19.

Experiment with different classifiers, trying to maximize the validation accuracy. You only need to show one classifier."

### Getting the cats_and_dogs data

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2019-04-17 00:26:51--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  91.2MB/s    in 0.7s    

2019-04-17 00:26:57 (91.2 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [0]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

## Problem 1

Do feature extraction with data augmentation.

In [5]:
from keras.applications import VGG19

conv_base = VGG19(
    weights='imagenet', 
    include_top=False, 
    input_shape=(150, 150, 3))

Instructions for updating:
Colocations handled automatically by placer.
80142336/80134624 [==============================] - 2s 0us/step


In [0]:
conv_base.trainable = False

In [8]:
from keras import layers
from keras import models
from keras import optimizers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
# no dropout
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 22,122,049
Trainable params: 2,097,665
Non-trainable params: 20,024,384
_________________________________________________________________


In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

# data augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

# compile model

model.compile(
    loss='binary_crossentropy', 
    optimizer=optimizers.RMSprop(lr=2e-5), 
    metrics=['acc'])

# train

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
100/100 [==============================] - 23s 228ms/step - loss: 0.6255 - acc: 0.6510 - val_loss: 0.5217 - val_acc: 0.7540
Epoch 2/30
100/100 [==============================] - 18s 175ms/step - loss: 0.5274 - acc: 0.7525 - val_loss: 0.4411 - val_acc: 0.8110
Epoch 3/30
100/100 [==============================] - 19s 190ms/step - loss: 0.4864 - acc: 0.7755 - val_loss: 0.3985 - val_acc: 0.8220
Epoch 4/30
100/100 [==============================] - 18s 180ms/step - loss: 0.4583 - acc: 0.7920 - val_loss: 0.3641 - val_acc: 0.8400
Epoch 5/30
100/100 [==============================] - 18s 177ms/step - loss: 0.4221 - acc: 0.8195 - val_loss: 0.3471 - val_acc: 0.8420
Epoch 6/30
100/100 [==============================] - 19s 190ms/step - loss: 0.4162 - acc: 0.8160 - val_loss: 0.3383 - val_acc: 0.8480
Epoch 7/30
100/100 [==============================] - 18s 

## Problem 2

Do fine-tuning with data augmentation.

In [14]:
conv_base = VGG19(
    weights='imagenet', 
    include_top=False, 
    input_shape=(150, 150, 3))

conv_base.trainable = False

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
#model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten_5 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 22,122,049
Trainable params: 2,097,665
Non-trainable params: 20,024,384
_________________________________________________________________


In [15]:
# data augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

# compile model

model.compile(
    loss='binary_crossentropy', 
    optimizer=optimizers.RMSprop(lr=2e-5), 
    metrics=['acc'])

# train

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 [==============================] - 20s 197ms/step - loss: 0.6205 - acc: 0.6610 - val_loss: 0.4895 - val_acc: 0.8050
Epoch 2/30
100/100 [==============================] - 17s 174ms/step - loss: 0.5187 - acc: 0.7660 - val_loss: 0.4174 - val_acc: 0.8180
Epoch 3/30
100/100 [==============================] - 17s 175ms/step - loss: 0.4664 - acc: 0.7835 - val_loss: 0.3719 - val_acc: 0.8390
Epoch 4/30
100/100 [==============================] - 18s 176ms/step - loss: 0.4350 - acc: 0.8075 - val_loss: 0.3473 - val_acc: 0.8500
Epoch 5/30
100/100 [==============================] - 19s 190ms/step - loss: 0.4254 - acc: 0.8075 - val_loss: 0.3411 - val_acc: 0.8540
Epoch 6/30
100/100 [==============================] - 18s 176ms/step - loss: 0.4054 - acc: 0.8270 - val_loss: 0.3175 - val_acc: 0.8630
Epoch 7/30
100/100 [==============================] - 17s 175ms/step - loss: 0.3935 - acc: 0.8250 - val_lo

### Fine-tuning

In [17]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
    
    
# compile model

model.compile(
    loss='binary_crossentropy',
    #
    # choose a smaller learning rate
    #
    optimizer=optimizers.RMSprop(lr=1e-5), 
    metrics=['acc'])

# train

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/100
100/100 [==============================] - 21s 206ms/step - loss: 0.3204 - acc: 0.8605 - val_loss: 0.2909 - val_acc: 0.8910
Epoch 2/100
100/100 [==============================] - 19s 190ms/step - loss: 0.2717 - acc: 0.8800 - val_loss: 0.2370 - val_acc: 0.8940
Epoch 3/100
100/100 [==============================] - 19s 188ms/step - loss: 0.2491 - acc: 0.8895 - val_loss: 0.1876 - val_acc: 0.9260
Epoch 4/100
100/100 [==============================] - 18s 184ms/step - loss: 0.2314 - acc: 0.9095 - val_loss: 0.2213 - val_acc: 0.9150
Epoch 5/100
100/100 [==============================] - 18s 183ms/step - loss: 0.2142 - acc: 0.9065 - val_loss: 0.2447 - val_acc: 0.9140
Epoch 6/100
100/100 [==============================] - 18s 183ms/step - loss: 0.2092 - acc: 0.9170 - val_loss: 0.1766 - val_acc: 0.9390
Epoch 7/100
100/100 [==============================] - 19s 194ms/step - loss: 0.2038 - acc: 0.9175 - val_loss: 0.2121 - val_acc: 0.9160
Epoch 8/100
100/100 [===========================

## Problem 3

Visualize heatmaps of class activation for the the model obtained in Problem 2.

In [0]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions

## Problem 4

Build an activation model that takes as input an image and produces as output the activation of the last conv layer of the model obtained in Problem 2. Using this activation model obtain the corresponding activations for the validation images. Apply t-SNE visualization to these activations to see how well the convnet separates cats from dogs.